# 数据读取与处理

In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

dataPath = 'C:/Users/zhang/mine/finTech/2021/data/A'

trainDf = pd.read_csv(dataPath + '/train_v1.csv')
wkdDf = pd.read_csv(dataPath + '/wkd_v1.csv')
testDf = pd.read_csv(dataPath + '/test_v1_periods.csv')
testDayDf = pd.read_csv(dataPath + '/test_v1_day.csv')

wkdDf = wkdDf.rename(columns={'ORIG_DT':'date'})

######### 这里数据官方已清洗过
trainDf = trainDf.merge(wkdDf, on = ['date'], how = 'left')
testDf = testDf.merge(wkdDf, on = ['date'], how = 'left')
testDayDf = testDayDf.merge(wkdDf, on = ['date'], how = 'left')

trainDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 695520 entries, 0 to 695519
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        695520 non-null  object
 1   post_id     695520 non-null  object
 2   biz_type    695520 non-null  object
 3   periods     695520 non-null  int64 
 4   amount      695520 non-null  int64 
 5   WKD_TYP_CD  695520 non-null  object
dtypes: int64(2), object(4)
memory usage: 37.1+ MB


In [2]:
## 提取岗位以日为粒度的业务量
def get_jobDayAmount(df):
    outputDf = df.groupby(['date', 'post_id', 'WKD_TYP_CD'], as_index = False)['amount'].sum()
    # outputDf = outputDf.sort_values(by = ['date', 'post_id'], axis = 0, ascending = True)
    return outputDf

trainDayDf = get_jobDayAmount(trainDf)
trainDayDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2070 entries, 0 to 2069
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        2070 non-null   object
 1   post_id     2070 non-null   object
 2   WKD_TYP_CD  2070 non-null   object
 3   amount      2070 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 80.9+ KB


In [3]:
## 提取岗位以0.5小时为粒度的业务量
def get_jobPeriodAmount(df):
    outputDf = df.groupby(['date', 'post_id', 'periods', 'WKD_TYP_CD'], as_index = False)['amount'].sum()
    # outputDf = outputDf.sort_values(by = ['date', 'post_id', 'periods'], axis = 0, ascending = True)
    return outputDf

trainDf = get_jobPeriodAmount(trainDf)
trainDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99360 entries, 0 to 99359
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        99360 non-null  object
 1   post_id     99360 non-null  object
 2   periods     99360 non-null  int64 
 3   WKD_TYP_CD  99360 non-null  object
 4   amount      99360 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 4.5+ MB


In [4]:
## 提取时间特征，此处以年、月、日作为变量
def getDateDf(df):
    df['date'] = pd.to_datetime(df['date'], format = '%Y/%m/%d')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df.drop(['date'], axis=1, inplace=True)
    return df

trainDf = getDateDf(trainDf)
trainDayDf = getDateDf(trainDayDf)
testDf = getDateDf(testDf)
testDayDf = getDateDf(testDayDf)
trainDf.head()

,post_id,periods,WKD_TYP_CD,amount,year,month,day
0,A,1,NH,0,2018,1,1
1,A,2,NH,0,2018,1,1
2,A,3,NH,0,2018,1,1
3,A,4,NH,0,2018,1,1
4,A,5,NH,0,2018,1,1


# 模型部分

In [5]:
## 载入所需库
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,accuracy_score, \
                            precision_score,recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [6]:
## 将字符向量转化为值向量
def labelEncoder_df(df, features):
    for i in features:
        encoder = LabelEncoder()
        df[i] = encoder.fit_transform(df[i])
        
## 节省内存
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## 以0.5h为粒度

In [7]:
trainCols = trainDf.columns.tolist()
testCols = testDf.columns.tolist()

In [8]:
trainDf['isTest'] = -1
testDf['isTest'] = 1
totalDf = pd.concat([trainDf, testDf])

In [9]:
cols = ['post_id', 'WKD_TYP_CD']
for col in cols:
    if totalDf[col].dtype == 'object':
        totalDf[col] = totalDf[col].astype(str)
labelEncoder_df(totalDf, cols)

In [10]:
trainDf = totalDf[totalDf['isTest'] == -1]
trainDf = trainDf[trainCols]
testDf = totalDf[totalDf['isTest'] == 1]
testDf = testDf[testCols]

trainDf['amount'] = trainDf['amount'].astype(int)

trainDf = reduce_mem_usage(trainDf)
testDf = reduce_mem_usage(testDf)

Mem. usage decreased to  1.61 Mb (67.3% reduction)
Mem. usage decreased to  0.06 Mb (58.9% reduction)


In [11]:
trainDf.info()
trainDf.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99360 entries, 0 to 99359
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   post_id     99360 non-null  int8 
 1   periods     99360 non-null  int8 
 2   WKD_TYP_CD  99360 non-null  int8 
 3   amount      99360 non-null  int16
 4   year        99360 non-null  int16
 5   month       99360 non-null  int8 
 6   day         99360 non-null  int8 
dtypes: int16(2), int8(5)
memory usage: 1.6 MB


,post_id,periods,WKD_TYP_CD,amount,year,month,day
0,0,1,0,0,2018,1,1
1,0,2,0,0,2018,1,1
2,0,3,0,0,2018,1,1
3,0,4,0,0,2018,1,1
4,0,5,0,0,2018,1,1


## 模型

In [12]:
dfX = trainDf.drop(['amount'], axis = 1)
dfY = trainDf['amount']

trainX, valX, trainY, valY = train_test_split(dfX, dfY, test_size=0.2, random_state = 42)    # random_state=42

In [13]:
################
## 评价函数 
def calMAPE(y, y_predict, name=None):
    y = y.to_numpy()
    y_predict = y_predict.to_numpy()
    temp = np.abs((y - y_predict) / y)
    temp = temp[~np.isinf(temp)]
    MAPE = np.nanmean(temp, axis=0)
    # MAPE = np.mean(np.abs((y - y_predict) / y)) * 100
    if name:
        print(name,':')
    print(f'MAPE is: {MAPE}')
    print('- - - - - - ')
    return MAPE

## 以0.5h为粒度的task2

In [14]:
random_seed = 2021
np.random.seed(2)
    
rf_cfl = RandomForestRegressor()
rf_cfl.fit(trainX, trainY)
y_rf = rf_cfl.predict(valX)
y_rf = y_rf.astype(int)
y_rf = pd.Series(y_rf.tolist())
y_rf.name = 'amount'
print("Train Score:%f" % rf_cfl.score(trainX, trainY))
print("Val Score:%f" % rf_cfl.score(valX, valY))

Train Score:0.997496
Val Score:0.983704


In [15]:
MAPE2 = calMAPE(valY, y_rf)
print(MAPE2)

MAPE is: 0.30204258992321253
- - - - - - 
0.30204258992321253


## 直接基于以0.5h为粒度的模型预测task1

In [16]:
valDayDf = pd.concat([valX, valY], axis = 1)
valDayDf = valDayDf.reset_index(drop = True)
valDayDf.head()

,post_id,periods,WKD_TYP_CD,year,month,day,amount
0,0,37,3,2018,1,4,1
1,0,23,3,2019,12,25,0
2,1,34,3,2019,7,29,372
3,0,44,3,2019,8,5,0
4,0,46,1,2020,7,12,0


In [17]:
rfDayDf = pd.concat([valX.reset_index(drop = True), y_rf], axis = 1)
rfDayDf = rfDayDf.reset_index(drop = True)
rfDayDf.head()

,post_id,periods,WKD_TYP_CD,year,month,day,amount
0,0,37,3,2018,1,4,0
1,0,23,3,2019,12,25,1279
2,1,34,3,2019,7,29,338
3,0,44,3,2019,8,5,0
4,0,46,1,2020,7,12,0


In [18]:
from datetime import datetime
def repairDate(df):
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['date'] = df['date'].apply(lambda x: datetime.strftime(x, format = '%Y/%#m/%#d'))
    df.drop(['year', 'month', 'day'], axis = 1, inplace = True)
    return df

In [19]:
valDayDf = repairDate(valDayDf)
valDayDf.head()

,post_id,periods,WKD_TYP_CD,amount,date
0,0,37,3,1,2018/1/4
1,0,23,3,0,2019/12/25
2,1,34,3,372,2019/7/29
3,0,44,3,0,2019/8/5
4,0,46,1,0,2020/7/12


In [20]:
rfDayDf = repairDate(rfDayDf)
rfDayDf.head()

,post_id,periods,WKD_TYP_CD,amount,date
0,0,37,3,0,2018/1/4
1,0,23,3,1279,2019/12/25
2,1,34,3,338,2019/7/29
3,0,44,3,0,2019/8/5
4,0,46,1,0,2020/7/12


In [21]:
valDayDf = get_jobDayAmount(valDayDf)
rfDayDf = get_jobDayAmount(rfDayDf)

In [22]:
valDayDf.head()

,date,post_id,WKD_TYP_CD,amount
0,2018/1/1,0,0,590
1,2018/1/1,1,0,0
2,2018/1/10,0,3,9454
3,2018/1/10,1,3,785
4,2018/1/11,0,3,6419


In [23]:
rfDayDf.head()

,date,post_id,WKD_TYP_CD,amount
0,2018/1/1,0,0,651
1,2018/1/1,1,0,0
2,2018/1/10,0,3,9426
3,2018/1/10,1,3,793
4,2018/1/11,0,3,6809


In [24]:
MAPE1 = calMAPE(valDayDf['amount'], rfDayDf['amount'])
print(MAPE1)

MAPE is: 0.10319184838559373
- - - - - - 
0.10319184838559373


In [25]:
MAPE = 0.7 * MAPE1 + 0.3 * MAPE2
print('total MAPE is:', MAPE)

total MAPE is: 0.16284707084687935


## Logistic

In [26]:
from sklearn.linear_model import LinearRegression, LogisticRegression

## 以0.5h为粒度的task2

In [27]:
random_seed = 2021
np.random.seed(2)
    
logit_cfl = LogisticRegression()
logit_cfl.fit(trainX, trainY)
y_logit = logit_cfl.predict(valX)
y_logit = y_logit.astype(int)
y_logit = pd.Series(y_logit.tolist())
y_logit.name = 'amount'
print("Train Score:%f" % logit_cfl.score(trainX, trainY))
print("Val Score:%f" % logit_cfl.score(valX, valY))

Train Score:0.631378
Val Score:0.631240


In [28]:
MAPE2 = calMAPE(valY, y_logit)
print(MAPE2)

MAPE is: 1.0
- - - - - - 
1.0
